In [1]:
import os
import sys

os.environ['PYSPARK_PYTHON'] = sys.executable
os.environ['PYSPARK_DRIVER_PYTHON'] = sys.executable

In [2]:
import findspark
findspark.init(r'D:\Spark\spark-3.5.1-bin-hadoop3')

In [3]:


import pyspark
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName("pyspark_ejercicio").getOrCreate()
spark

In [4]:
from pyspark.sql.functions import col

In [5]:
%%time
data = spark.read.csv(path= 'Data/Diabetes.csv',header=True,inferSchema=True)

CPU times: total: 0 ns
Wall time: 17.3 s


In [6]:
%%time
data.printSchema()

root
 |-- Diabetes_012: double (nullable = true)
 |-- HighBP: integer (nullable = true)
 |-- HighChol: double (nullable = true)
 |-- CholCheck: integer (nullable = true)
 |-- BMI: double (nullable = true)
 |-- Smoker: double (nullable = true)
 |-- Stroke: double (nullable = true)
 |-- HeartDiseaseorAttack: double (nullable = true)
 |-- PhysActivity: integer (nullable = true)
 |-- Fruits: integer (nullable = true)
 |-- Veggies: integer (nullable = true)
 |-- HvyAlcoholConsump: integer (nullable = true)
 |-- AnyHealthcare: integer (nullable = true)
 |-- NoDocbcCost: double (nullable = true)
 |-- GenHlth: double (nullable = true)
 |-- MentHlth: double (nullable = true)
 |-- PhysHlth: double (nullable = true)
 |-- DiffWalk: double (nullable = true)
 |-- Sex: integer (nullable = true)
 |-- Age: integer (nullable = true)
 |-- Education: double (nullable = true)
 |-- Income: double (nullable = true)

CPU times: total: 0 ns
Wall time: 3.99 ms


In [7]:
data.toPandas().isna().sum()

Diabetes_012            0
HighBP                  0
HighChol                0
CholCheck               0
BMI                     0
Smoker                  0
Stroke                  0
HeartDiseaseorAttack    0
PhysActivity            0
Fruits                  0
Veggies                 0
HvyAlcoholConsump       0
AnyHealthcare           0
NoDocbcCost             0
GenHlth                 0
MentHlth                0
PhysHlth                0
DiffWalk                0
Sex                     0
Age                     0
Education               0
Income                  0
dtype: int64

0 = no diabetes, 1 = prediabetes, 2 = diabetes

In [8]:
data.groupBy("Diabetes_012").count().show()

+------------+------+
|Diabetes_012| count|
+------------+------+
|         0.0|197191|
|         1.0|  5619|
|         2.0| 33568|
+------------+------+



In [9]:
data.filter(col('Diabetes_012') == 0).show()

+------------+------+--------+---------+----+------+------+--------------------+------------+------+-------+-----------------+-------------+-----------+-------+--------+--------+--------+---+---+---------+------+
|Diabetes_012|HighBP|HighChol|CholCheck| BMI|Smoker|Stroke|HeartDiseaseorAttack|PhysActivity|Fruits|Veggies|HvyAlcoholConsump|AnyHealthcare|NoDocbcCost|GenHlth|MentHlth|PhysHlth|DiffWalk|Sex|Age|Education|Income|
+------------+------+--------+---------+----+------+------+--------------------+------------+------+-------+-----------------+-------------+-----------+-------+--------+--------+--------+---+---+---------+------+
|         0.0|     0|     1.0|        1|15.0|   1.0|   0.0|                 0.0|           0|     1|      1|                0|            1|        0.0|    5.0|    10.0|    20.0|     0.0|  0| 11|      4.0|   5.0|
|         0.0|     0|     0.0|        1|24.0|   1.0|   0.0|                 0.0|           0|     0|      0|                0|            1|        

Vamos hacer clustering y a analizar los resultados

In [10]:

from pyspark.ml.feature import VectorAssembler

In [11]:
vector_assembler = VectorAssembler()
vector_assembler.explainParam("handleInvalid")

"handleInvalid: How to handle invalid data (NULL and NaN values). Options are 'skip' (filter out rows with invalid data), 'error' (throw an error), or 'keep' (return relevant number of NaN in the output). Column lengths are taken from the size of ML Attribute Group, which can be set using `VectorSizeHint` in a pipeline before `VectorAssembler`. Column lengths can also be inferred from first rows of the data since it is safe to do so but only in case of 'error' or 'skip'). (default: error)"

In [12]:
print(vector_assembler.explainParams())

handleInvalid: How to handle invalid data (NULL and NaN values). Options are 'skip' (filter out rows with invalid data), 'error' (throw an error), or 'keep' (return relevant number of NaN in the output). Column lengths are taken from the size of ML Attribute Group, which can be set using `VectorSizeHint` in a pipeline before `VectorAssembler`. Column lengths can also be inferred from first rows of the data since it is safe to do so but only in case of 'error' or 'skip'). (default: error)
inputCols: input column names. (undefined)
outputCol: output column name. (default: VectorAssembler_5a1cfb01b047__output)


In [13]:
help(VectorAssembler)

Help on class VectorAssembler in module pyspark.ml.feature:

class VectorAssembler(pyspark.ml.wrapper.JavaTransformer, pyspark.ml.param.shared.HasInputCols, pyspark.ml.param.shared.HasOutputCol, pyspark.ml.param.shared.HasHandleInvalid, pyspark.ml.util.JavaMLReadable, pyspark.ml.util.JavaMLWritable)
 |  VectorAssembler(*, inputCols: Optional[List[str]] = None, outputCol: Optional[str] = None, handleInvalid: str = 'error')
 |  
 |  A feature transformer that merges multiple columns into a vector column.
 |  
 |  .. versionadded:: 1.4.0
 |  
 |  Examples
 |  --------
 |  >>> df = spark.createDataFrame([(1, 0, 3)], ["a", "b", "c"])
 |  >>> vecAssembler = VectorAssembler(outputCol="features")
 |  >>> vecAssembler.setInputCols(["a", "b", "c"])
 |  VectorAssembler...
 |  >>> vecAssembler.transform(df).head().features
 |  DenseVector([1.0, 0.0, 3.0])
 |  >>> vecAssembler.setParams(outputCol="freqs").transform(df).head().freqs
 |  DenseVector([1.0, 0.0, 3.0])
 |  >>> params = {vecAssembler.inp

In [14]:
assembler = VectorAssembler(inputCols = data.columns,
                            outputCol = "features",
                            handleInvalid='skip')

data = assembler.transform(data)

In [15]:
data.select('features').show(5,truncate=False)

+--------------------------------------------------------------------------------------------------+
|features                                                                                          |
+--------------------------------------------------------------------------------------------------+
|(22,[2,3,4,5,9,10,12,14,15,16,19,20,21],[1.0,1.0,15.0,1.0,1.0,1.0,1.0,5.0,10.0,20.0,11.0,4.0,5.0])|
|(22,[0,1,3,4,7,9,12,14,19,20,21],[2.0,1.0,1.0,28.0,1.0,1.0,1.0,2.0,11.0,4.0,3.0])                 |
|[2.0,1.0,1.0,1.0,33.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,1.0,0.0,2.0,10.0,0.0,0.0,0.0,9.0,4.0,7.0]       |
|[2.0,0.0,1.0,1.0,29.0,0.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,0.0,5.0,0.0,30.0,1.0,1.0,12.0,3.0,4.0]      |
|(22,[3,4,5,12,14,17,18,19,20,21],[1.0,24.0,1.0,1.0,3.0,1.0,1.0,13.0,5.0,6.0])                     |
+--------------------------------------------------------------------------------------------------+
only showing top 5 rows



In [16]:
from pyspark.ml.feature import StandardScaler

In [17]:
scaler = StandardScaler()


In [18]:
print(scaler.explainParams())

inputCol: input column name. (undefined)
outputCol: output column name. (default: StandardScaler_8f368e746c37__output)
withMean: Center data with mean (default: False)
withStd: Scale to unit standard deviation (default: True)


In [19]:
scaler = StandardScaler(inputCol = "features",
                        outputCol = "scaled_features",
                        withStd = True,
                        withMean = False)

data = scaler.fit(data).transform(data)



In [20]:
data.select(col('scaled_features')).show(5,truncate=False)

+----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|scaled_features                                                                                                                                                                                                                                                                                                                               |
+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [21]:
data.count()

236378

Primero vamos a probar con Kmeans a ver que tal nos salen los clusters

In [22]:
from pyspark.ml.clustering import KMeans


In [23]:
modelo  = KMeans()

In [24]:
print(modelo.explainParams())

distanceMeasure: the distance measure. Supported options: 'euclidean' and 'cosine'. (default: euclidean)
featuresCol: features column name. (default: features)
initMode: The initialization algorithm. This can be either "random" to choose random points as initial cluster centers, or "k-means||" to use a parallel variant of k-means++ (default: k-means||)
initSteps: The number of steps for k-means|| initialization mode. Must be > 0. (default: 2)
k: The number of clusters to create. Must be > 1. (default: 2)
maxBlockSizeInMB: maximum memory in MB for stacking input data into blocks. Data is stacked within partitions. If more than remaining data size in a partition then it is adjusted to the data size. Default 0.0 represents choosing optimal value, depends on specific algorithm. Must be >= 0. (default: 0.0)
maxIter: max number of iterations (>= 0). (default: 20)
predictionCol: prediction column name. (default: prediction)
seed: random seed. (default: 4645095874236920005)
solver: The solver 

In [25]:
kmeans = KMeans(distanceMeasure='euclidean',
                featuresCol='scaled_features',
                initMode= "k-means||",
                initSteps= 1,
                k= 3,
                maxIter=50,
                predictionCol="prediccion",
                seed= 12345678910,
                solver='block',
                tol= 0.0001,
                weightCol='Diabetes_012')

model = kmeans.fit(data)



In [26]:
cluster = model.transform(data)

In [27]:
print(cluster.select(col('Diabetes_012'),col('prediccion')).show())

+------------+----------+
|Diabetes_012|prediccion|
+------------+----------+
|         0.0|         2|
|         2.0|         1|
|         2.0|         0|
|         2.0|         2|
|         0.0|         1|
|         0.0|         2|
|         0.0|         1|
|         2.0|         0|
|         0.0|         1|
|         0.0|         1|
|         2.0|         0|
|         2.0|         1|
|         2.0|         2|
|         2.0|         1|
|         0.0|         1|
|         0.0|         1|
|         0.0|         0|
|         0.0|         0|
|         1.0|         1|
|         0.0|         0|
+------------+----------+
only showing top 20 rows

None


In [28]:
from pyspark.ml.evaluation import ClusteringEvaluator

# Calcula la métrica de silueta
evaluator = ClusteringEvaluator(predictionCol='prediccion')
silhouette = evaluator.evaluate(cluster)

# Muestra el valor de la métrica de silueta
print("Silhouette Score:", silhouette)

Silhouette Score: 0.28243737439973304


In [29]:
kmeans = KMeans(distanceMeasure='euclidean',
                featuresCol='scaled_features',
                initMode= "k-means||",
                initSteps= 1,
                k= 3,
                maxIter=50,
                predictionCol="prediccion",
                seed= 12345678910,
                solver='block',
                tol= 0.0001,
                weightCol='Diabetes_012')

model = kmeans.fit(data)
cluster = model.transform(data)



In [30]:
def c_kmeans(distance, init, initS, maxIt, solv, data):
    kmeans = KMeans(distanceMeasure=distance,
                    featuresCol='scaled_features',
                    initMode=init,
                    initSteps=initS,
                    k=3,
                    maxIter=maxIt,
                    predictionCol="prediccion",
                    seed=12345678910,
                    solver=solv,
                    tol=0.0001,
                    weightCol='Diabetes_012')

    model = kmeans.fit(data)
    
    # Calcula la métrica de silueta
    evaluator = ClusteringEvaluator(predictionCol='prediccion')
    silhouette = evaluator.evaluate(model.transform(data))
    
    return silhouette


In [31]:
print(modelo.explainParams())

distanceMeasure: the distance measure. Supported options: 'euclidean' and 'cosine'. (default: euclidean)
featuresCol: features column name. (default: features)
initMode: The initialization algorithm. This can be either "random" to choose random points as initial cluster centers, or "k-means||" to use a parallel variant of k-means++ (default: k-means||)
initSteps: The number of steps for k-means|| initialization mode. Must be > 0. (default: 2)
k: The number of clusters to create. Must be > 1. (default: 2)
maxBlockSizeInMB: maximum memory in MB for stacking input data into blocks. Data is stacked within partitions. If more than remaining data size in a partition then it is adjusted to the data size. Default 0.0 represents choosing optimal value, depends on specific algorithm. Must be >= 0. (default: 0.0)
maxIter: max number of iterations (>= 0). (default: 20)
predictionCol: prediction column name. (default: prediction)
seed: random seed. (default: 4645095874236920005)
solver: The solver 

In [32]:
# def c_kmeans(distanceMeasure,initMode,initSteps,maxIter,solver):


In [33]:
print(c_kmeans('cosine','k-means||',2,1,'row',data))

0.30299709537434366


In [34]:
print(c_kmeans('cosine','k-means||',2,1,'block',data))

0.30299709537434366


In [35]:
print(c_kmeans('euclidean','k-means||',2,1,'block',data))

0.325973305587922


In [36]:
print(c_kmeans('euclidean','k-means||',2,1,'block',data))

0.325973305587922


Bisecting K-Means


In [37]:
data2 = spark.read.csv(path= 'Data/Diabetes.csv',header=True,inferSchema=True)

In [38]:
from pyspark.ml.clustering import BisectingKMeans

In [39]:
mod = BisectingKMeans()

In [40]:
print(mod.explainParams())

distanceMeasure: the distance measure. Supported options: 'euclidean' and 'cosine'. (default: euclidean)
featuresCol: features column name. (default: features)
k: The desired number of leaf clusters. Must be > 1. (default: 4)
maxIter: max number of iterations (>= 0). (default: 20)
minDivisibleClusterSize: The minimum number of points (if >= 1.0) or the minimum proportion of points (if < 1.0) of a divisible cluster. (default: 1.0)
predictionCol: prediction column name. (default: prediction)
seed: random seed. (default: -3433244116123162505)
weightCol: weight column name. If this is not set or empty, we treat all instance weights as 1.0. (undefined)


In [52]:
assembler = VectorAssembler(inputCols = data2.columns,
                            outputCol = "features",
                            handleInvalid='skip')

data2 = assembler.transform(data2)

In [54]:
scaler = StandardScaler(inputCol = "features",
                        outputCol = "scaled_features",
                        withStd = True,
                        withMean = False)

data2 = scaler.fit(data2).transform(data2)


In [55]:


bkmeans = BisectingKMeans(featuresCol    = "scaled_features",
                         predictionCol   = "bkcluster", 
                         k               = 3,
                         distanceMeasure = "euclidean",
                         seed = 12345678910
                         
                         )

model = bkmeans.fit(data2)

In [56]:
# Realizar predicciones en los datos
predictions = model.transform(data2)

# Calcular la inercia
evaluator = ClusteringEvaluator(predictionCol='bkcluster')
inertia = evaluator.evaluate(predictions)

print("Inertia:", inertia)

Inertia: -0.015275344248975885


Con bkmeans no me salen bien la inercia de los clusters voy a probar hacer el modelo de clasificación

In [57]:
from pyspark.ml.classification import DecisionTreeClassifier

In [88]:
data = spark.read.csv(path= 'Data/Diabetes.csv',header=True,inferSchema=True)

In [60]:
train, test = data.randomSplit(weights = [0.7, 0.3], seed = 42)

In [82]:
from pyspark.ml.feature import VectorAssembler, StandardScaler
from pyspark.ml.classification import DecisionTreeClassifier
from pyspark.ml import Pipeline
train, test = data.randomSplit(weights = [0.7, 0.3], seed = 42)
# Ensamblar las características en un vector denso
assembler = VectorAssembler(inputCols=data.columns, outputCol="raw_features")

# Normalizar las características
scaler = StandardScaler(inputCol="raw_features", outputCol="features", withStd=True, withMean=False)

# Dividir en conjuntos de entrenamiento y prueba
train, test = data.randomSplit(weights=[0.7, 0.3], seed=42)

# Definir el clasificador de árbol de decisión
dtc = DecisionTreeClassifier(featuresCol="features", labelCol="Diabetes_012", predictionCol="prediction", maxDepth=2)

# Crear el pipeline
pipeline = Pipeline(stages=[assembler, scaler, dtc])

# Entrenar el modelo
model = pipeline.fit(train)

# Hacer predicciones en el conjunto de prueba
predictions = model.transform(test)



In [72]:
predictions.filter(col('Diabetes_012')==1.0).select(col('Diabetes_012'),col('prediction')).show(50)

+------------+----------+
|Diabetes_012|prediction|
+------------+----------+
|         1.0|       1.0|
|         1.0|       1.0|
|         1.0|       1.0|
|         1.0|       1.0|
|         1.0|       1.0|
|         1.0|       1.0|
|         1.0|       1.0|
|         1.0|       1.0|
|         1.0|       1.0|
|         1.0|       1.0|
|         1.0|       1.0|
|         1.0|       1.0|
|         1.0|       1.0|
|         1.0|       1.0|
|         1.0|       1.0|
|         1.0|       1.0|
|         1.0|       1.0|
|         1.0|       1.0|
|         1.0|       1.0|
|         1.0|       1.0|
|         1.0|       1.0|
|         1.0|       1.0|
|         1.0|       1.0|
|         1.0|       1.0|
|         1.0|       1.0|
|         1.0|       1.0|
|         1.0|       1.0|
|         1.0|       1.0|
|         1.0|       1.0|
|         1.0|       1.0|
|         1.0|       1.0|
|         1.0|       1.0|
|         1.0|       1.0|
|         1.0|       1.0|
|         1.0|       1.0|
|         1.

In [83]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

# Crear un evaluador de clasificación multiclase
evaluator = MulticlassClassificationEvaluator(labelCol="Diabetes_012", predictionCol="prediction", metricName="accuracy")

# Calcular la precisión en el conjunto de predicciones
accuracy = evaluator.evaluate(predictions)

# Imprimir la precisión
print("Accuracy:", accuracy)

Accuracy: 1.0


In [80]:
evaluator = MulticlassClassificationEvaluator(labelCol = "Diabetes_012",
                                              predictionCol = "prediction",
                                              metricName = "f1")

f1 = evaluator.evaluate(predictions)

print(f1)

1.0


In [89]:

assembler = VectorAssembler(inputCols=data.columns, outputCol="features")
data = assembler.transform(data)
train, test = data.randomSplit(weights=[0.7, 0.3], seed=42)

# Definir el clasificador de árbol de decisión
dtc = DecisionTreeClassifier(featuresCol='features', labelCol="Diabetes_012", predictionCol="prediction", maxDepth=2)

model = dtc.fit(train)

y_hat = model.transform(test)




In [91]:
# Crear un evaluador de clasificación multiclase
evaluator = MulticlassClassificationEvaluator(labelCol="Diabetes_012", predictionCol="prediction", metricName="accuracy")

# Calcular la precisión en el conjunto de predicciones
accuracy = evaluator.evaluate(y_hat)

# Imprimir la precisión
print("Accuracy:", accuracy)

Accuracy: 1.0
